In [1]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import pandas as pd

import glob

import os
import random
from scipy.spatial import distance

import nltk

import re

try to flatten the axon data and then do a heatmap

### Normalise neurons to left hemisphere

In [18]:
# need to check if some is in the left or right hemisphere if its in the left then proceed to next brain if its in the 
# righ update all original files to be in the left

path = os.getcwd()

list_of_files = glob.glob(str(path)+'/ENT1807141/*neurites.xlsx') # get a list of filepaths of files of neuron points

# generate .swc files (brainrender doesnt recognise endpoints and branching points so need to change those to regular axon/dendrite points) 
def create_swc_file (neuron_dataset):
    neuron_soma_axon = neuron_dataset
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(5, 2)
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(6, 2)
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(7, 3)
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(11, 3)
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(8, 4)
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(12, 4)
    neuron_soma_axon_final = neuron_soma_axon.iloc[:, [0, 1, 3, 4, 5, 6, 7]]
    return neuron_soma_axon_final


for file in list_of_files:
    neuron = pd.read_excel(file) # go through each of the neuron files
    file_name_root = file.split("/")[-1].replace("_neurites.xlsx","") # get the neuron ID (mouseID + neuron number)
    soma_coordinates = neuron[neuron["Ntype"] == 1] #soma coordinates are filtered
    z_soma = soma_coordinates.iloc[0,5] # extract the z coordinate (mediolateral axis) of the soma
  
    if z_soma < 227.7799187764: # if z coordinate is past midline then mirror all the neuron points
        neuron["z"] = 455.5598375528-neuron["z"]
        neuron.to_excel(file, index=False) # regenerate the neuron files with updated coordiinates

        file_path = str(os.path.dirname(file)) + "/" + file_name_root + ".swc" # update the neuron swc files with updated coordiinates
        swc_file = create_swc_file(neuron)
        swc_file.to_csv(file_path, sep=" ", index=False, header=False)
        
    else:
        continue

    

406.4236576
good 1807141_7
405.5937152
good 1807141_15
404.6486743528
good 1807141_38
405.8987064
good 1807141_16
398.7252071528
good 1807141_36
379.4222823528
good 1807141_32
404.05516
good 1807141_25
398.617424
good 1807141_22
406.5046832
good 1807141_26
395.7908879528
good 1807141_31
404.79292
good 1807141_8
403.8045456
good 1807141_24
394.592392
good 1807141_19
401.7084335528001
good 1807141_37
398.1415648
good 1807141_20
405.9501968
good 1807141_13
404.7631408
good 1807141_17
404.3585919528001
good 1807141_39
405.638976
good 1807141_10
406.6115056
good 1807141_14


![title](info/Ntype.png)

### Filter the files to only have axon data

In [23]:
# function for craeting the file with just axons and soma
def create_swc_file (neuron_file_original):
    neuron_original = pd.read_excel(neuron_file_original)
    soma_axon = [1,2,5,6]
    neuron_soma_axon = neuron_original[neuron_original["Ntype"].isin(soma_axon)]
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(5, 2)
    neuron_soma_axon.loc[:, "Ntype"] = neuron_soma_axon["Ntype"].replace(6, 2)
    neuron_soma_axon_final = neuron_soma_axon.iloc[:, [0, 1, 3, 4, 5, 6, 7]]
    return neuron_soma_axon_final

# need a list of all neurite.xlsx files 
path = os.getcwd()
list_of_files = glob.glob(str(path)+'/ENT*/*neurites.xlsx') 


for file_name in list_of_files:
      file_path = str(file_name).replace("_neurites.xlsx","_axon_only.swc")
      print(file_path)
      create_swc_file(file_name).to_csv(file_path, sep=" ", index=False, header=False) #generate swc files with axons only

['/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_2_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_41_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_6_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_26_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_31_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_35_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_22_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_32_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_25_neurites.xlsx', '/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_18_neurite

### Get cortical endpoint-only files for all neurons

In [ ]:

path = os.getcwd()

list_of_files = glob.glob(str(path)+'/ENT*/*neurites.xlsx') 

# generate endpoint files
for file_name in list_of_files:
    neuron_data = pd.read_excel(file_name)
    endpoint_coordinates = neuron_data[neuron_data["Ntype"] == 6]
    file_name_root = file_name.split("/")[-1].replace("_neurites.xlsx","")
    endpoint_coordinates.to_csv(file_name_root + "axon_endpoints.csv", index=False)


### Master list of all endpoint counts and axon length

In [2]:

from pathlib import Path

import pandas as pd

import glob

import os

visual_areas = [
    "VIS1", "VIS2/3", "VIS4", "VIS5", "VIS6a", "VIS6b", 
    "VISal", "VISam", "VISl1", "VISl2/3", "VISl4", "VISl5" , "VISl6",
    "VISp1", "VISp2/3", "VISp4", "VISp5", "VISp6", "VISpm", 
    "VISli"
]

somatomotor_areas = [
    "MO1", "MO2/3", "MO5", "MO6a", "MO6b", "MOp", "MOs"
]

hippocampus = ["HPF", "CA1", "CA2", "CA3", "DG"]

STRdorsal = ["CP"]

STRventral = ["ACB", "FS", "OT" , "STR" ]

amygdala = ["LA","BLA","BMA"]

Posterior_parietal_association_areas = ["VISrl","VISa1", "VISa2/3", "VISa4", "VISa5", "VISa6"]


# the new dataset will have rows as different neurons and collumns as different areas with their endpoint count
columns = [
    "neuron_ID", 
    "FRP", "MO1", "MO2/3", "MO5", "MO6a", "MO6b", "MOp", "MOs", "SS", "GU", "VISC", "AUD", 
    "VIS1", "VIS2/3", "VIS4", "VIS5", "VIS6a", "VIS6b", 
    "VISal", "VISam", "VISl1", "VISl2/3", "VISl4", "VISl5" , "VISl6",
    "VISp1", "VISp2/3", "VISp4", "VISp5", "VISp6",
    "VISpm", 
    "VISli", "VISpor", "VISpl",
    "ACA", "PL", "ILA", "ORB", "AI", "RSP", "VISa1", "VISa2/3", "VISa4", "VISa5", "VISa6", 
    "VISrl", "TEa",
    "PERI", "ECT", "OLF", "HPF", "CA1", "CA2", "CA3", "DG", "PAR", "POST", "PRE", "SUB", 
    "ACB", "FS", "OT" , "STR", "CP", "CLA", "EPv", "EPd", "LA", "BLA","BMA"
]

master_list_endpoints=pd.DataFrame(columns=columns)

brain_areas = [
   "FRP", "MO1", "MO2/3", "MO5", "MO6a", "MO6b", "MOp", "MOs", "SS", "GU", "VISC", "AUD", 
    "VIS1", "VIS2/3", "VIS4", "VIS5", "VIS6a", "VIS6b", 
    "VISal", "VISam", "VISl1", "VISl2/3", "VISl4", "VISl5" , "VISl6",
    "VISp1", "VISp2/3", "VISp4", "VISp5", "VISp6",
    "VISpm", 
    "VISli", "VISpor", "VISpl",
    "ACA", "PL", "ILA", "ORB", "AI", "RSP", "VISa1", "VISa2/3", "VISa4", "VISa5", "VISa6", 
    "VISrl", "TEa",
    "PERI", "ECT", "OLF", "HPF", "CA1", "CA2", "CA3", "DG", "PAR", "POST", "PRE", "SUB", 
    "ACB", "FS", "OT" , "STR", "CP", "CLA", "EPv", "EPd", "LA", "BLA","BMA"
]

path = os.getcwd()

list_of_brain_folders = glob.glob(str(path) + '/ENT*/') # all the forlders must start with ENT and then the brain ID



for folder in list_of_brain_folders:
    list_of_neuron_endings = glob.glob(folder + "*neurites.xlsx")
    
    for neuron in list_of_neuron_endings:
       
       ID = os.path.basename(neuron).replace("_neurites.xlsx","")
       print(ID) 
       endpoint_info = pd.read_excel(neuron)
       endpoint_info = endpoint_info[endpoint_info["Ntype"] == 6]
       target_endpoint_list = []
       for brain_area in brain_areas:
            target_strenght = endpoint_info['Abbreviation'].str.startswith(brain_area).sum()
            target_endpoint_list.append(target_strenght) 
        # add all into a list
    
       new_row = pd.DataFrame([[ID, *target_endpoint_list]], columns = columns)
       master_list_endpoints = pd.concat([master_list_endpoints, new_row])
        

master_list_endpoints['VIS'] = master_list_endpoints[visual_areas].sum(axis=1)
master_list_endpoints['MO'] = master_list_endpoints[somatomotor_areas].sum(axis=1)
master_list_endpoints['HIP'] = master_list_endpoints[hippocampus].sum(axis=1)
master_list_endpoints['STRd'] = master_list_endpoints[STRdorsal].sum(axis=1)
master_list_endpoints['STRv'] = master_list_endpoints[STRventral].sum(axis=1)
master_list_endpoints['AMY'] = master_list_endpoints[amygdala].sum(axis=1)
master_list_endpoints["PTLp"] = master_list_endpoints[Posterior_parietal_association_areas].sum(axis=1)
master_list_endpoints["EP"] = master_list_endpoints[["EPd","EPv"]].sum(axis=1)
areas_to_drop = visual_areas + somatomotor_areas + hippocampus + STRdorsal + STRventral + amygdala + Posterior_parietal_association_areas + ["EPd", "EPv"]


master_list_endpoints.drop(columns=areas_to_drop, inplace=True)

master_list_endpoints = master_list_endpoints[["neuron_ID", "FRP", "SS", "VIS", "VISpor", "RSP", "PTLp", "ORB", "MO", "AUD",
    "ACA", "PL", "ILA", "VISC", "AI", "GU", "TEa", "PERI", "ECT", "OLF", "PAR", 
    "POST", "PRE", "SUB", "HIP", "CLA", "EP", "AMY", "STRd", "STRv"]]


# ---------------axon length in each area---------------------


master_list_length=pd.DataFrame(columns=columns)


for folder in list_of_brain_folders:
    list_of_neuron_length = glob.glob(folder + "*length.xlsx")
    
    for neuron in list_of_neuron_length:
       
       ID = os.path.basename(neuron).replace("_length.xlsx","") 
       print(ID) 
       length_info = pd.read_excel(neuron)
       print("Columns in file:", length_info.columns.tolist())
       target_length_list = []
       for brain_area in brain_areas:
            if length_info['Abbreviation'].str.startswith(brain_area).sum() > 0:
                filtered_df = length_info[length_info['Abbreviation'].str.startswith(brain_area)]
                # Sum the "Length" column in the filtered DataFrame
                total_length = filtered_df["Length"].sum()
                target_length_list.append(total_length) # add all into a list
            else:
                target_length_list.append(0)
       new_row = pd.DataFrame([[ID, *target_length_list]], columns = columns)
       master_list_length = pd.concat([master_list_length, new_row])
        

master_list_length['VIS'] = master_list_length[visual_areas].sum(axis=1)
master_list_length['MO'] = master_list_length[somatomotor_areas].sum(axis=1)
master_list_length['HIP'] = master_list_length[hippocampus].sum(axis=1)
master_list_length['STRd'] = master_list_length[STRdorsal].sum(axis=1)
master_list_length['STRv'] = master_list_length[STRventral].sum(axis=1)
master_list_length['AMY'] = master_list_length[amygdala].sum(axis=1)
master_list_length["PTLp"] = master_list_length[Posterior_parietal_association_areas].sum(axis=1)
master_list_length["EP"] = master_list_length[["EPd","EPv"]].sum(axis=1)
areas_to_drop = visual_areas + somatomotor_areas + hippocampus + STRdorsal + STRventral + amygdala + Posterior_parietal_association_areas + ["EPd", "EPv"]


master_list_length.drop(columns=areas_to_drop, inplace=True)

master_list_length = master_list_length[["neuron_ID", "FRP", "SS", "VIS", "VISpor", "RSP", "PTLp", "ORB", "MO", "AUD",
    "ACA", "PL", "ILA", "VISC", "AI", "GU", "TEa", "PERI", "ECT", "OLF", "PAR", 
    "POST", "PRE", "SUB", "HIP", "CLA", "EP", "AMY", "STRd", "STRv"]]




# merging of lenth and endpoint data into a maste file

df_length_renamed = master_list_length.rename(
    columns={col: f"{col}_length" for col in master_list_length.columns if col != "neuron_ID"}
)

 
df_endpoint_renamed = master_list_endpoints.rename(
    columns={col: f"{col}_endpoint" for col in master_list_endpoints.columns if col != "neuron_ID"}
)

merged_df = pd.merge(df_endpoint_renamed, df_length_renamed, on="neuron_ID")


cols = merged_df.columns
id_col = ["neuron_ID"]
endpoint_cols = [col for col in cols if col.endswith("_endpoint")]
length_cols = [col for col in cols if col.endswith("_length")]

# Interleave columns
interleaved_cols = id_col + [c for pair in zip(endpoint_cols, length_cols) for c in pair]
merged_df = merged_df[interleaved_cols]



master_list_file_path = str(path)+"/master_list_ALL_BRAIN_ALL_AREAS.xlsx"
print(master_list_file_path)
merged_df.to_excel(master_list_file_path, index=False)
 


1056_2
1056_41
1056_6
1056_26
1056_31
1056_35
1056_22
1056_32
1056_25
1056_18
1056_21
1056_36
1056_1
1056_5
1056_12
1056_16
1056_38
1056_11
1056_28
1056_15
1056_8
1056_14
1056_29
1056_10
1056_9
1056_39
1056_17
1056_13
1056_37
1056_20
1056_19
1056_24
1056_33
1056_4
1056_7
1056_40
1056_3
1056_23
1056_34
1056_30
1056_27
1057_16
1057_12
1057_8
1057_15
1057_11
1057_1
1057_5
1057_2
1057_18
1057_6
1057_7
1057_19
1057_3
1057_4
1057_10
1057_14
1057_9
1057_13
1057_17
1807141_7
1807141_15
1807141_38
1807141_16
1807141_36
1807141_32
1807141_25
1807141_22
1807141_26
1807141_31
1807141_8
1807141_24
1807141_19
1807141_37
1807141_20
1807141_13
1807141_17
1807141_39
1807141_10
1807141_14
1814612_8
1814612_6
1814612_5
1814612_22
1814612_4
1814612_23
1814612_9
1814612_13
1814612_17
1814612_14
1849931_12
1849931_4
1849931_7
1849931_3
1849931_9
1849931_6
1849931_10
1849931_14
1849931_13
1849931_5
1135_16
1135_5
1135_15
1135_8
1135_9
1135_14
1135_7
1135_13
1135_17
1056_28
Columns in file: ['ID', 'Length', '

/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
/var/folders/k5/pxqlmzvj061dk9ld1lj8nv000000gn/T/ipykernel_34805/3464740447.py:129: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  master_list_length = pd.concat([master_list_length, new_row])
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not s

Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_15
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_27
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_10
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_22
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_19
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_37
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_2
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_40
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullNa

/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be re

Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_20
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_9
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_35
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_29
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_3
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_18
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_36
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1056_23
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullNam

...

Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_25
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_17
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_39
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_20
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_8
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_36
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_14
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1807141_26
Columns in file: ['ID', 'Length',

/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be re

Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1814612_5
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1814612_22
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1814612_13
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1814612_6
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1849931_3
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1849931_14
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1849931_6
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1849931_5
Columns in file: ['ID', 'Length', 'Ab

/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
/opt/miniconda3/envs/neuron_morph/lib/python3.12/site-packages/openpyxl/worksheet/_read_only.py:85: UserWarning: Unknown extension is not supported and will be re

Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_15
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_7
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_13
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_8
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_16
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_14
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_17
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName', 'totalLength_um', 'Abbreviation.1', 'Length.1']
1135_9
Columns in file: ['ID', 'Length', 'Abbreviation', 'FullName

### All soma coordinates

In [19]:
path = os.getcwd()

list_of_files = glob.glob(str(path)+'/ENT*/*neurites.xlsx') 
print(len(list_of_files))

soma_df = pd.DataFrame(columns=["ID","x", "y", "z"]) # new dataset for soma coordinates

for file_name in list_of_files:
    neuron_data = pd.read_excel(file_name) 
    soma_coordinates = neuron_data[neuron_data["Ntype"] == 1] #soma coordinates are filtered
    x_soma = soma_coordinates.iloc[0,3] * 25 # scale to 25um
    y_soma = soma_coordinates.iloc[0,4] * 25
    z_soma = soma_coordinates.iloc[0,5] * 25    
    ID = file_name.split("/")[-1].replace("_neurites.xlsx","")
    soma = {"ID": ID, "x": x_soma, "y": y_soma, "z": z_soma}
    soma_df.loc[len(soma_df)] = soma # add a new row at the end of the dataset

    

soma_df.to_csv(str(path) + "/all_soma_coordinates.csv", sep=",", index=False, header=True)

109
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_2_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_41_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_6_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_26_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_31_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_35_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_22_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_32_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_25_neurites.xlsx
/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/ENT1056/1056_18_neurites.xlsx
/Users/atenejonaus

### Drop soma of incomplete neurons

In [20]:
incomplete_neuron_ids = [
    "1057_19",
    "1057_14",
    "1056_22",
    "1056_34",
    "1057_12",
    "1807141_24",
    "1807141_19",
    "1807141_22",
    "1056_14",
    "1056_20",
    "1056_23",
    "1056_28",
    "1056_38",
    "1056_39",
    "1056_40",
    "1057_5",
    "1057_10",
    "1057_11",
    "1057_15",
    "1057_18",
    "1135_8",
    "1135_15",
    "1849931_3"
]
soma_file = pd.read_csv("/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/all_soma_coordinates.csv")

soma_filtered = soma_file[~soma_file["ID"].isin(incomplete_neuron_ids)]

soma_filtered.to_csv("/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/all_soma_coordinates_filtered.csv", sep=",", index=False, header=True)



In [38]:
master_list = pd.read_csv("/Users/atenejonauskyte/Downloads/master_list_endpoints_length.csv")
soma_list = pd.read_csv("/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/all_soma_coordinates.csv")


joint_master_soma = pd.merge(master_list, soma_list, how="left", on="ID")
print(joint_master_soma)

joint_master_soma.to_csv("/Users/atenejonauskyte/EC_project_Image_analysis/single_neuron_data/soma_master_list.csv", sep=",", index=False, header=True)

    mouse_ID  cell_ID          ID injection EC position  FRP_endpoint  \
0       1135       13     1135_13       RSC         NaN             0   
1       1135        7      1135_7       RSC         NaN             0   
2       1135        9      1135_9       RSC         NaN             0   
3    1849931       12  1849931_12       RSC         NaN             0   
4    1849931        4   1849931_4       RSC         NaN             0   
..       ...      ...         ...       ...         ...           ...   
80      1056       13     1056_13       ORB      int EC             0   
81      1056       33     1056_33       ORB                         0   
82      1057        1      1057_1       ORB                         2   
83   1807141       10  1807141_10       ORB         NaN             1   
84   1807141       13  1807141_13       ORB         NaN             0   

     FRP_length  SS_endpoint     SS_length  VIS_endpoint  ...  AMY_endpoint  \
0      0.000000            0      0.000000  